In [1]:
# Importing Required Packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)# To see all the columns of a dataframe
#pd.set_option('display.max_rows', None)

In [2]:
# Loading reduced feature training set
X_train = pd.read_csv('X_train_final.csv')
y_train = pd.read_csv('y_train.final.csv')

In [3]:
# Loading reduced feature test set
X_test = pd.read_csv('X_test_final.csv')
y_test = pd.read_csv('y_test.final.csv')

### Standardizing all the columns, including dummy variables, for SELU activated Dense model

In [4]:
# Importing the Standard scaler from Sklearn
from sklearn.preprocessing import StandardScaler

In [5]:
# Instantiating the Standard Scaler object & fit_transforming the reduced feature training set.
st_scaler = StandardScaler() 
arr = st_scaler.fit_transform(X_train)

In [6]:
# Converting the array to the corresponding dataframe
X_train_scaled = pd.DataFrame(arr, columns=X_train.columns, index=X_train.index)

In [7]:
# Transforming the test set with the same standard scaler object as was used to fit Training set.
arr1 = st_scaler.transform(X_test)

In [8]:
# Converting the array to the corresponding dataframe.
X_test_scaled = pd.DataFrame(arr1, columns=X_test.columns, index=X_test.index)

## Model_11: Keras Dense Model with Equal Neurons/layer, SELU Activation & Tuned learning rate.

In [9]:
# Importing Optuna , Tensorflow & Keras
import optuna , tensorflow as tf

In [10]:
# Importing Keras
from tensorflow import keras

In [11]:
# Importing the Sklearn's roc_auc_score module
from sklearn.metrics import roc_auc_score

In [12]:
# Importing train test split from Sklearn to produce validation set
from sklearn.model_selection import train_test_split

In [13]:
# Performing train_validation split with test size = .15 to provide more Training data to Neural Net
X_train_1, X_val, y_train_1, y_val = train_test_split(X_train_scaled, y_train, test_size=0.15, random_state=42,
                                                          stratify=y_train)

In [14]:
# defining class weight dictionary list for the imbalanced class
cl_weight = [{0:1.0,1:11.38697},{0:1.0,1:12.0},{0:1.0,1:13.0},{0:1.0,1:14.0},{0:1.0,1:15.0},{0:1.0,1:16.0},
             {0:1.0,1:17.0},{0:1.0,1:18.0}]

In [15]:
# Defining best roc_auc score
best_roc_auc = 0

In [16]:
# defining the objective function
def objective_wrappper_keras_eq(X_tr, y_tr, X_val, y_val, cl_w):
    '''
    Optimizes Keras ,neural net with equal no. of neurons in each layer , best parameters 
    on the given training set: X_tr, y_tr using validation set: X_val,y_val & class weights: cl_w
    
    '''

    def objective(trial):
        s = trial.suggest_int('step',2,6)
        def exponential_decay_fn(epoch):
            return 0.01 * 0.1**(epoch /s )
              
        # building model & searching for the best no. of neural units per layer.
        no_units = trial.suggest_int('no._units',100,600)# No. of Neurons in each layer
        model = keras.models.Sequential()
        model.add(keras.layers.InputLayer(input_shape=X_tr.shape[1:]))
        #model.add(keras.layers.BatchNormalization())
        for i in range(trial.suggest_int('number',2,6)):
            model.add(keras.layers.Dense(units=no_units,activation='selu',
                                     kernel_initializer='lecun_normal')) # For self normalization
        model.add(keras.layers.Dense(units=1,activation='sigmoid'))
    
        # Searching for the best learning rate.
        #lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True) 
    
        # Compiling the model
        model.compile(loss="binary_crossentropy", optimizer=keras.optimizers.Nadam(beta_1=0.9, beta_2=0.999),
                      metrics=[tf.keras.metrics.AUC(curve="ROC")])
    
        # Defining the Callbacks
        checkpoint_cb = keras.callbacks.ModelCheckpoint('best_model.h5',save_best_only=True) # 1st Callback
        early_stopping_cb = keras.callbacks.EarlyStopping(patience=4) # 2nd Callback, Stop if validation score doen't improve for
        # 4 epochs        
        lr_scheduler_cb = keras.callbacks.LearningRateScheduler(exponential_decay_fn)# 3rd Callback
        #tensorboard_cb = keras.callbacks.TensorBoard(get_run_logdir()) 
        
        #Fitting the model
        model.fit(X_tr, y_tr, epochs=50, validation_data=(X_val,y_val), batch_size=32,
             class_weight=trial.suggest_categorical('best_weight',cl_w),
             callbacks=[checkpoint_cb, early_stopping_cb, lr_scheduler_cb])#,tensorboard_cb])
    
        # Loading the best model 
        model_best = keras.models.load_model('best_model.h5')
    
        # Computing the roc_auc_score for the validation set
        roc_auc_1 = roc_auc_score(y_val,model_best.predict_proba(X_val))
        global best_roc_auc
        # Updating best_roc_auc score
        if roc_auc_1 > best_roc_auc:
            best_roc_auc = roc_auc_1
            model_best.save('Best_model_Selu_eq_Learn.h5')
        
        return roc_auc_1
    
    return objective

In [17]:
# Defining the evaluation function for study's best parameters
def study_best_score_params(X_tr, y_tr, X_v, y_v, obj_func, cl_w, n_trials=100):
    ''' Computes the best hyper parameters of the Neural net using the Training set(X_tr,y_tr) & Validation set
    (X_v,y_v), class weights cl_w & 
    returns Optuna's study's best score & clasifier parameters'''
    
    study = optuna.create_study(direction='maximize')
    study.optimize(obj_func(X_tr, y_tr, X_v, y_v, cl_w),n_trials)
    best_score = study.best_value
    best_params = study.best_params
    return (best_score,best_params)

In [18]:
# Extracting the best model parameters and best study score
best_study_score,best_study_params = study_best_score_params(X_train_1, y_train_1, X_val, y_val, objective_wrappper_keras_eq, cl_weight,
                                                            n_trials=50)

[I 2021-01-05 00:24:14,144] A new study created in memory with name: no-name-0b21dd42-5b01-4387-8f6e-aa5633f7ef4d


  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train on 196038 samples, validate on 34595 samples
Epoch 1/50
196038/196038 [==============================] - 25s 130us/sample - loss: 4.6109 - auc: 0.5025 - val_loss: 1.5189 - val_auc: 0.5000
Epoch 2/50
196038/196038 [==============================] - 24s 124us/sample - loss: 1.5165 - auc: 0.4988 - val_loss: 1.5090 - val_auc: 0.5000
Epoch 3/50
196038/196038 [==============================] - 24s 123us/sample - loss: 1.4531 - auc: 0.4985 - val_loss: 1.5077 - val_auc: 0.5000
Epoch 4/50
196038/196038 [==============================] - 24s 124us/sample - loss: 1.4273 - auc: 0.5028 - val_loss: 1.5155 - val_auc: 0.5000
Epoch 5/50
196038/196038 [==============================] - 24s 124us/sample - loss: 1.4171 - auc: 0.4971 - val_loss: 1.4245 - val_auc: 0.5000
Epoch 6/50
196038/196038 [==============================] - 24s 124us/sample - loss: 1.4120 - auc: 0.5016 - val_loss: 1.4099 - val_auc: 0.5000
Epoch 7/50
196038/196038 [===============

[I 2021-01-05 00:31:12,389] Trial 0 finished with value: 0.5 and parameters: {'step': 2, 'no._units': 568, 'number': 5, 'best_weight': {0: 1.0, 1: 14.0}}. Best is trial 0 with value: 0.5.


  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train on 196038 samples, validate on 34595 samples
Epoch 1/50
196038/196038 [==============================] - 18s 93us/sample - loss: 4.0905 - auc_1: 0.6125 - val_loss: 2.9738 - val_auc_1: 0.7284
Epoch 2/50
196038/196038 [==============================] - 17s 88us/sample - loss: 22.3165 - auc_1: 0.6574 - val_loss: 2.2696 - val_auc_1: 0.7352
Epoch 3/50
196038/196038 [==============================] - 17s 88us/sample - loss: 1.4057 - auc_1: 0.7111 - val_loss: 3.1440 - val_auc_1: 0.7388
Epoch 4/50
196038/196038 [==============================] - 17s 88us/sample - loss: 1.4802 - auc_1: 0.7175 - val_loss: 2.0030 - val_auc_1: 0.7491
Epoch 5/50
196038/196038 [==============================] - 17s 88us/sample - loss: 1.3450 - auc_1: 0.7356 - val_loss: 2.2615 - val_auc_1: 0.7481
Epoch 6/50
196038/196038 [==============================] - 17s 88us/sample - loss: 1.3003 - auc_1: 0.7504 - val_loss: 1.9329 - val_auc_1: 0.7490
Epoch 7/50
196038/1960

[I 2021-01-05 00:34:24,887] Trial 1 finished with value: 0.7511888420414058 and parameters: {'step': 6, 'no._units': 543, 'number': 2, 'best_weight': {0: 1.0, 1: 16.0}}. Best is trial 1 with value: 0.7511888420414058.


...

[I 2021-01-05 03:20:41,926] Trial 47 finished with value: 0.5013786634013858 and parameters: {'step': 2, 'no._units': 169, 'number': 3, 'best_weight': {0: 1.0, 1: 17.0}}. Best is trial 34 with value: 0.7560548910166114.


  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train on 196038 samples, validate on 34595 samples
Epoch 1/50
196038/196038 [==============================] - 17s 85us/sample - loss: 1.7097 - auc_48: 0.6617 - val_loss: 1.8028 - val_auc_48: 0.7254
Epoch 2/50
196038/196038 [==============================] - 16s 81us/sample - loss: 1.3167 - auc_48: 0.7232 - val_loss: 1.5523 - val_auc_48: 0.7437
Epoch 3/50
196038/196038 [==============================] - 16s 80us/sample - loss: 1.2666 - auc_48: 0.7431 - val_loss: 1.3944 - val_auc_48: 0.7471
Epoch 4/50
196038/196038 [==============================] - 16s 80us/sample - loss: 1.2327 - auc_48: 0.7585 - val_loss: 1.2690 - val_auc_48: 0.7524
Epoch 5/50
196038/196038 [==============================] - 16s 80us/sample - loss: 1.2071 - auc_48: 0.7698 - val_loss: 1.2567 - val_auc_48: 0.7535
Epoch 6/50
196038/196038 [==============================] - 16s 80us/sample - loss: 1.1904 - auc_48: 0.7769 - val_loss: 1.2795 - val_auc_48: 0.7526
Epoch 7/50


[I 2021-01-05 03:23:05,694] Trial 48 finished with value: 0.7535045658113768 and parameters: {'step': 3, 'no._units': 182, 'number': 2, 'best_weight': {0: 1.0, 1: 15.0}}. Best is trial 34 with value: 0.7560548910166114.


  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train on 196038 samples, validate on 34595 samples
Epoch 1/50
196038/196038 [==============================] - 17s 89us/sample - loss: 1.5389 - auc_49: 0.6746 - val_loss: 1.4092 - val_auc_49: 0.7389
Epoch 2/50
196038/196038 [==============================] - 16s 83us/sample - loss: 1.2659 - auc_49: 0.7263 - val_loss: 1.2620 - val_auc_49: 0.7453
Epoch 3/50
196038/196038 [==============================] - 16s 83us/sample - loss: 1.2205 - auc_49: 0.7455 - val_loss: 1.6851 - val_auc_49: 0.7400
Epoch 4/50
196038/196038 [==============================] - 16s 83us/sample - loss: 1.1900 - auc_49: 0.7594 - val_loss: 1.2085 - val_auc_49: 0.7559
Epoch 5/50
196038/196038 [==============================] - 16s 83us/sample - loss: 1.1641 - auc_49: 0.7696 - val_loss: 1.2206 - val_auc_49: 0.7562
Epoch 6/50
196038/196038 [==============================] - 16s 83us/sample - loss: 1.1489 - auc_49: 0.7761 - val_loss: 1.2157 - val_auc_49: 0.7561
Epoch 7/50


[I 2021-01-05 03:25:18,202] Trial 49 finished with value: 0.7558950844097945 and parameters: {'step': 3, 'no._units': 142, 'number': 2, 'best_weight': {0: 1.0, 1: 14.0}}. Best is trial 34 with value: 0.7560548910166114.


In [19]:
print('The best roc_auc_score for the study is: ',best_study_score)

The best roc_auc_score for the study is:  0.7560548910166114


In [20]:
print('The best study parameters for the classifier are: ',best_study_params)

The best study parameters for the classifier are:  {'step': 2, 'no._units': 423, 'number': 2, 'best_weight': {0: 1.0, 1: 14.0}}


In [21]:
# loading the best Neural Network
Best_Model_selu = keras.models.load_model('Best_model_Selu_eq_Learn.h5')

In [22]:
# Printing its summary
Best_Model_selu.summary()

Model: "sequential_34"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_134 (Dense)            (None, 423)               43992     
_________________________________________________________________
dense_135 (Dense)            (None, 423)               179352    
_________________________________________________________________
dense_136 (Dense)            (None, 1)                 424       
Total params: 223,768
Trainable params: 223,768
Non-trainable params: 0
_________________________________________________________________


In [23]:
# Defining the function to calculate the roc_auc score for the feature sets
def cal_roc_auc(X, y, cls, f_set, t_set, model_name):
    ''' Calculates the roc auc score using the best study parameters 
        f_set : String: specifies 'full feature', 'Reduced feature'
        t_set: String: specifies 'training', 'test'
        model_name: String: specifies Name of the model '''
        
    y_pred = cls.predict_proba(X)
    print('The roc_auc_score for the {} {} set using the best {} classifier is '.format(f_set,t_set,model_name),roc_auc_score(y,y_pred))

In [24]:
# Calculating the Reduced feature training set roc_auc score using the pretrained Neural Network having 
# best study parameters & weights
cal_roc_auc(X_train_scaled, y_train, Best_Model_selu, 'Reduced feature', 'training', 'Dense Neural Network using SELU activation')

The roc_auc_score for the Reduced feature training set using the best Dense Neural Network using SELU activation classifier is  0.7735250280260082


In [25]:
# Calculating the Reduced feature test set roc_auc score using the pretrained Neural Network having 
# best study parameters & weights
cal_roc_auc(X_test_scaled, y_test, Best_Model_selu, 'Reduced feature', 'test', 'Dense Neural Network using SELU activation')

The roc_auc_score for the Reduced feature test set using the best Dense Neural Network using SELU activation classifier is  0.757445351657831


## Observation(s):
### 1) Though the tuned Selu dense neural classifier has test set roc_auc score of 0.757445351657831, with almost no overfitting, certain optima trials have ended with roc_auc score of .5, corresponding to random guessing. It may be that dataset is allergic to Nadam optimizer or Selu sctivation function. thus let's discard this model for this dataset.

## Model_11.1: Keras Dense Model with Equal Neurons/layer, SELU Activation, RMS Prop Optimizer & Tuned learning rate.

In [33]:
# Defining best roc_auc score
best_roc_auc = 0

In [34]:
# defining the objective function
def objective_wrappper_keras_eq_RMS(X_tr, y_tr, X_val, y_val, cl_w):
    '''
    Optimizes Keras ,neural net with equal no. of neurons in each layer , best parameters 
    on the given training set: X_tr, y_tr using validation set: X_val,y_val & class weights: cl_w
    
    '''

    def objective(trial):
        s = trial.suggest_int('step',2,6)
        def exponential_decay_fn(epoch):
            return 0.01 * 0.1**(epoch /s )
              
        # building model & searching for the best no. of neural units per layer.
        no_units = trial.suggest_int('no._units',100,600)# No. of Neurons in each layer
        model = keras.models.Sequential()
        model.add(keras.layers.InputLayer(input_shape=X_tr.shape[1:]))
        #model.add(keras.layers.BatchNormalization())
        for i in range(trial.suggest_int('number',2,6)):
            model.add(keras.layers.Dense(units=no_units,activation='selu',
                                     kernel_initializer='lecun_normal')) # For self normalization
        model.add(keras.layers.Dense(units=1,activation='sigmoid'))
    
        # Searching for the best learning rate.
        #lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True) 
    
        # Compiling the model
        model.compile(loss="binary_crossentropy", optimizer=keras.optimizers.RMSprop(),
                      metrics=[tf.keras.metrics.AUC(curve="ROC")])
        
        # Defining the Callbacks
        checkpoint_cb = keras.callbacks.ModelCheckpoint('best_model.h5',save_best_only=True) # 1st Callback
        early_stopping_cb = keras.callbacks.EarlyStopping(patience=4,restore_best_weights=True) # 2nd Callback, Stop if validation score doen't improve for
        # 4 epochs        
        lr_scheduler_cb = keras.callbacks.LearningRateScheduler(exponential_decay_fn)# 3rd Callback
        #tensorboard_cb = keras.callbacks.TensorBoard(get_run_logdir()) 
        
        #Fitting the model
        model.fit(X_tr, y_tr, epochs=50, validation_data=(X_val,y_val), batch_size=32,
             class_weight=trial.suggest_categorical('best_weight',cl_w),
             callbacks=[checkpoint_cb, early_stopping_cb, lr_scheduler_cb])#,tensorboard_cb])
    
        # Loading the best model 
        model_best = keras.models.load_model('best_model.h5')
    
        # Computing the roc_auc_score for the validation set
        roc_auc_1 = roc_auc_score(y_val,model_best.predict_proba(X_val))
        global best_roc_auc
        # Updating best_roc_auc score
        if roc_auc_1 > best_roc_auc:
            best_roc_auc = roc_auc_1
            model_best.save('Best_model_Selu_eq_Learn_RMS.h5')
        
        return roc_auc_1
    
    return objective

In [35]:
# Defining the evaluation function for study's best parameters
def study_best_score_params(X_tr, y_tr, X_v, y_v, obj_func, cl_w, n_trials=100):
    ''' Computes the best hyper parameters of the Neural net using the Training set(X_tr,y_tr) & Validation set
    (X_v,y_v), class weights cl_w & 
    returns Optuna's study's best score & clasifier parameters'''
    
    study = optuna.create_study(direction='maximize')
    study.optimize(obj_func(X_tr, y_tr, X_v, y_v, cl_w),n_trials)
    best_score = study.best_value
    best_params = study.best_params
    return (best_score,best_params)

In [36]:
# Extracting the best model parameters and best study score
best_study_score,best_study_params = study_best_score_params(X_train_1, y_train_1, X_val, y_val, objective_wrappper_keras_eq_RMS, cl_weight,
                                                            n_trials=10)

[I 2021-01-05 20:56:57,233] A new study created in memory with name: no-name-d7c2a11f-f359-46c1-89c6-60b330e460f7


  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train on 196038 samples, validate on 34595 samples
Epoch 1/50
196038/196038 [==============================] - 16s 84us/sample - loss: 9.4790 - auc_2: 0.5295 - val_loss: 29.4142 - val_auc_2: 0.5079
Epoch 2/50
196038/196038 [==============================] - 15s 79us/sample - loss: 3.5154 - auc_2: 0.5101 - val_loss: 3.8783 - val_auc_2: 0.5033
Epoch 3/50
196038/196038 [==============================] - 16s 79us/sample - loss: 2.2828 - auc_2: 0.5176 - val_loss: 3.3285 - val_auc_2: 0.5792
Epoch 4/50
196038/196038 [==============================] - 16s 80us/sample - loss: 1.8605 - auc_2: 0.5406 - val_loss: 1.9503 - val_auc_2: 0.5669
Epoch 5/50
196038/196038 [==============================] - 16s 80us/sample - loss: 1.6091 - auc_2: 0.6301 - val_loss: 1.4378 - val_auc_2: 0.7199
Epoch 6/50
196038/196038 [==============================] - 16s 80us/sample - loss: 1.4431 - auc_2: 0.6926 - val_loss: 1.4145 - val_auc_2: 0.7154
Epoch 7/50
196038/1960

[I 2021-01-05 21:00:39,010] Trial 0 finished with value: 0.7336060679157983 and parameters: {'step': 4, 'no._units': 394, 'number': 3, 'best_weight': {0: 1.0, 1: 15.0}}. Best is trial 0 with value: 0.7336060679157983.


  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train on 196038 samples, validate on 34595 samples
Epoch 1/50
196038/196038 [==============================] - 19s 95us/sample - loss: 2.3205 - auc_3: 0.5028 - val_loss: 1.6852 - val_auc_3: 0.5000
Epoch 2/50
196038/196038 [==============================] - 18s 89us/sample - loss: 1.5733 - auc_3: 0.4973 - val_loss: 2.3404 - val_auc_3: 0.5000
Epoch 3/50
196038/196038 [==============================] - 17s 89us/sample - loss: 1.4606 - auc_3: 0.4960 - val_loss: 1.8156 - val_auc_3: 0.5000
Epoch 4/50
196038/196038 [==============================] - 18s 89us/sample - loss: 1.3959 - auc_3: 0.5003 - val_loss: 1.8611 - val_auc_3: 0.5000
Epoch 5/50
196038/196038 [==============================] - 18s 90us/sample - loss: 1.3637 - auc_3: 0.4980 - val_loss: 1.4810 - val_auc_3: 0.5000
Epoch 6/50
196038/196038 [==============================] - 18s 90us/sample - loss: 1.3425 - auc_3: 0.4988 - val_loss: 1.3922 - val_auc_3: 0.5000
Epoch 7/50
196038/19603

[I 2021-01-05 21:07:28,566] Trial 1 finished with value: 0.5 and parameters: {'step': 6, 'no._units': 152, 'number': 6, 'best_weight': {0: 1.0, 1: 12.0}}. Best is trial 0 with value: 0.7336060679157983.


...

[I 2021-01-05 21:31:54,949] Trial 7 finished with value: 0.5 and parameters: {'step': 3, 'no._units': 387, 'number': 6, 'best_weight': {0: 1.0, 1: 12.0}}. Best is trial 0 with value: 0.7336060679157983.


  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train on 196038 samples, validate on 34595 samples
Epoch 1/50
196038/196038 [==============================] - 16s 82us/sample - loss: 2.9939 - auc_10: 0.5346 - val_loss: 2.3453 - val_auc_10: 0.4996
Epoch 2/50
196038/196038 [==============================] - 15s 76us/sample - loss: 1.7515 - auc_10: 0.5547 - val_loss: 2.0175 - val_auc_10: 0.5090
Epoch 3/50
196038/196038 [==============================] - 15s 77us/sample - loss: 1.4452 - auc_10: 0.6886 - val_loss: 1.8676 - val_auc_10: 0.7176
Epoch 4/50
196038/196038 [==============================] - 15s 76us/sample - loss: 1.4377 - auc_10: 0.7251 - val_loss: 1.4935 - val_auc_10: 0.7145
Epoch 5/50
196038/196038 [==============================] - 15s 76us/sample - loss: 1.4026 - auc_10: 0.7327 - val_loss: 1.4137 - val_auc_10: 0.7254
Epoch 6/50
196038/196038 [==============================] - 15s 76us/sample - loss: 1.3781 - auc_10: 0.7358 - val_loss: 1.4543 - val_auc_10: 0.7228
Epoch 7/50


[I 2021-01-05 21:34:42,042] Trial 8 finished with value: 0.7253752029908114 and parameters: {'step': 2, 'no._units': 202, 'number': 3, 'best_weight': {0: 1.0, 1: 16.0}}. Best is trial 0 with value: 0.7336060679157983.


  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train on 196038 samples, validate on 34595 samples
Epoch 1/50
196038/196038 [==============================] - 16s 82us/sample - loss: 3.1882 - auc_11: 0.5377 - val_loss: 3.5060 - val_auc_11: 0.5001
Epoch 2/50
196038/196038 [==============================] - 15s 76us/sample - loss: 2.1123 - auc_11: 0.5141 - val_loss: 4.4321 - val_auc_11: 0.5000
Epoch 3/50
196038/196038 [==============================] - 15s 77us/sample - loss: 1.7539 - auc_11: 0.5043 - val_loss: 2.5915 - val_auc_11: 0.5000
Epoch 4/50
196038/196038 [==============================] - 15s 77us/sample - loss: 1.7372 - auc_11: 0.5322 - val_loss: 2.1206 - val_auc_11: 0.4988
Epoch 5/50
196038/196038 [==============================] - 15s 77us/sample - loss: 1.6437 - auc_11: 0.5787 - val_loss: 1.4811 - val_auc_11: 0.6451
Epoch 6/50
196038/196038 [==============================] - 15s 77us/sample - loss: 1.5468 - auc_11: 0.6429 - val_loss: 1.4571 - val_auc_11: 0.6852
Epoch 7/50


[I 2021-01-05 21:37:15,059] Trial 9 finished with value: 0.6867356384528662 and parameters: {'step': 6, 'no._units': 215, 'number': 3, 'best_weight': {0: 1.0, 1: 15.0}}. Best is trial 0 with value: 0.7336060679157983.


## Observation(s):
### 1) Similar to the previous case, certain optima trials have again ended with roc_auc score of .5. It is almost surely now that SELU activation is not the right activation function for this dataset.

## Model_11.2: Keras Dense Model with Equal Neurons/layer, SELU Activation, Nesterov Optimizer & Tuned learning rate.

In [18]:
# Defining best roc_auc score
best_roc_auc = 0

In [19]:
# defining the objective function
def objective_wrappper_keras_eq_Nesterov(X_tr, y_tr, X_val, y_val, cl_w):
    '''
    Optimizes Keras ,neural net with equal no. of neurons in each layer , best parameters 
    on the given training set: X_tr, y_tr using validation set: X_val,y_val & class weights: cl_w
    
    '''

    def objective(trial):
        s = trial.suggest_int('step',2,6)
        def exponential_decay_fn(epoch):
            return 0.01 * 0.1**(epoch /s )
              
        # building model & searching for the best no. of neural units per layer.
        no_units = trial.suggest_int('no._units',100,600)# No. of Neurons in each layer
        model = keras.models.Sequential()
        model.add(keras.layers.InputLayer(input_shape=X_tr.shape[1:]))
        #model.add(keras.layers.BatchNormalization())
        for i in range(trial.suggest_int('number',2,6)):
            model.add(keras.layers.Dense(units=no_units,activation='selu',
                                     kernel_initializer='lecun_normal')) # For self normalization
        model.add(keras.layers.Dense(units=1,activation='sigmoid'))
    
        # Searching for the best learning rate.
        #lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True) 
    
        # Compiling the model
        model.compile(loss="binary_crossentropy", optimizer=keras.optimizers.SGD(momentum=.9,nesterov=True),
                      metrics=[tf.keras.metrics.AUC(curve="ROC")])
        
        # Defining the Callbacks
        checkpoint_cb = keras.callbacks.ModelCheckpoint('best_model.h5',save_best_only=True) # 1st Callback
        early_stopping_cb = keras.callbacks.EarlyStopping(patience=4,restore_best_weights=True) # 2nd Callback, Stop if validation score doen't improve for
        # 4 epochs        
        lr_scheduler_cb = keras.callbacks.LearningRateScheduler(exponential_decay_fn)# 3rd Callback
        #tensorboard_cb = keras.callbacks.TensorBoard(get_run_logdir()) 
        
        #Fitting the model
        model.fit(X_tr, y_tr, epochs=50, validation_data=(X_val,y_val), batch_size=32,
             class_weight=trial.suggest_categorical('best_weight',cl_w),
             callbacks=[checkpoint_cb, early_stopping_cb, lr_scheduler_cb])#,tensorboard_cb])
    
        # Loading the best model 
        model_best = keras.models.load_model('best_model.h5')
    
        # Computing the roc_auc_score for the validation set
        roc_auc_1 = roc_auc_score(y_val,model_best.predict_proba(X_val))
        global best_roc_auc
        # Updating best_roc_auc score
        if roc_auc_1 > best_roc_auc:
            best_roc_auc = roc_auc_1
            model_best.save('Best_model_Selu_eq_Learn_Nesterov.h5')
        
        return roc_auc_1
    
    return objective

In [20]:
# Defining the evaluation function for study's best parameters
def study_best_score_params(X_tr, y_tr, X_v, y_v, obj_func, cl_w, n_trials=100):
    ''' Computes the best hyper parameters of the Neural net using the Training set(X_tr,y_tr) & Validation set
    (X_v,y_v), class weights cl_w & 
    returns Optuna's study's best score & clasifier parameters'''
    
    study = optuna.create_study(direction='maximize')
    study.optimize(obj_func(X_tr, y_tr, X_v, y_v, cl_w),n_trials)
    best_score = study.best_value
    best_params = study.best_params
    return (best_score,best_params)

In [21]:
# Extracting the best model parameters and best study score
best_study_score,best_study_params = study_best_score_params(X_train_1, y_train_1, X_val, y_val, objective_wrappper_keras_eq_Nesterov, cl_weight,
                                                            n_trials=10)

[I 2021-01-07 21:57:57,299] A new study created in memory with name: no-name-690ac7a3-e37b-4305-9089-64ed7f5bb558


  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train on 196038 samples, validate on 34595 samples
Epoch 1/50
    32/196038 [..............................] - ETA: 1:27:21 - loss: 6.4671 - auc: 0.4349WARNING:tensorflow:Can save best model only with val_loss available, skipping.


[W 2021-01-07 21:57:59,200] Trial 0 failed because of the following error: InvalidArgumentError()
Traceback (most recent call last):
  File "C:\Users\panka\anaconda3\lib\site-packages\optuna\study.py", line 778, in _run_trial
    result = func(trial)
  File "<ipython-input-19-fcb6aa84e5fb>", line 41, in objective
    callbacks=[checkpoint_cb, early_stopping_cb, lr_scheduler_cb])#,tensorboard_cb])
  File "C:\Users\panka\anaconda3\lib\site-packages\tensorflow_core\python\keras\engine\training.py", line 819, in fit
    use_multiprocessing=use_multiprocessing)
  File "C:\Users\panka\anaconda3\lib\site-packages\tensorflow_core\python\keras\engine\training_v2.py", line 342, in fit
    total_epochs=epochs)
  File "C:\Users\panka\anaconda3\lib\site-packages\tensorflow_core\python\keras\engine\training_v2.py", line 128, in run_one_epoch
    batch_outs = execution_function(iterator)
  File "C:\Users\panka\anaconda3\lib\site-packages\tensorflow_core\python\keras\engine\training_v2_utils.py", line

InvalidArgumentError: 2 root error(s) found.
  (0) Invalid argument:  assertion failed: [predictions must be >= 0] [Condition x >= y did not hold element-wise:] [x (sequential/dense_6/Sigmoid:0) = ] [[nan][nan][nan]...] [y (metrics/auc/Cast_2/x:0) = ] [0]
	 [[{{node metrics/auc/assert_greater_equal/Assert/AssertGuard/else/_24/Assert}}]]
	 [[metrics/auc/assert_less_equal/Assert/AssertGuard/pivot_f/_36/_71]]
  (1) Invalid argument:  assertion failed: [predictions must be >= 0] [Condition x >= y did not hold element-wise:] [x (sequential/dense_6/Sigmoid:0) = ] [[nan][nan][nan]...] [y (metrics/auc/Cast_2/x:0) = ] [0]
	 [[{{node metrics/auc/assert_greater_equal/Assert/AssertGuard/else/_24/Assert}}]]
0 successful operations.
0 derived errors ignored. [Op:__inference_distributed_function_1619]

Function call stack:
distributed_function -> distributed_function


## Observation(s):
### 1) The above case of using SELU activation and a Non-Adaptive optimizer Nestorov, resulted in error during the very first trial, substantiating the foregone conclusion that SELU activation is indeed , not the right activation function for this dataset.